In [60]:
# We import the libraries that are needed
# for solving linear programming problems
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# For performance analysis purposes
from time import process_time

# For saving the results returned by the solver
from pandas import Series, DataFrame

# For better code readability and maintanability
from typing import Final

# Define the constants
NUMBER_OF_INTENSIVISTS: Final = 24
WEEK: Final = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
#WEEKDAYS: Final = WEEK[:5]
SHIFTS: Final = ['Mi', 'Mn', 'Ai', 'An', 'Ni', 'Nn']

In [61]:
# We define the continuous model
nonlinear_model = pyo.ConcreteModel()

# We create the labels of the 1st dimension of the continuous model
# that correspondes to the rows of our previously seen matrix
i_labels = SHIFTS

# We check that the the length of the 1st dimension
# is equal to 12
assert len(i_labels) == len(SHIFTS)

# We create the labels of the 2nd dimension of the continuous model
# that corresponds to the columns of our previously seen matrix
j_labels = WEEK

# We check the length of the 2nd dimension
# to be equal to 7
assert len(j_labels) == 5

# We initialize the model dimensions with the labels
# that were defined above
nonlinear_model.i = pyo.Set(initialize = i_labels, doc = "scenarios")
nonlinear_model.j = pyo.Set(initialize = j_labels, doc = "days of the week")

patient_forecast = {

    # In-Python used notation:
    # M <- morning shift; 
    # A <- afternoon shift;
    # N <- night shift;
    # i <- infectious;
    # n <- non-infectious

    ('Mi', 'Mon'): 25,
    ('Mi', 'Tue'): 22,
    ('Mi', 'Wed'): 18,
    ('Mi', 'Thu'): 23,
    ('Mi', 'Fri'): 30,
    
    ('Mn', 'Mon'): 15,
    ('Mn', 'Tue'): 8,
    ('Mn', 'Wed'): 10,
    ('Mn', 'Thu'): 8,
    ('Mn', 'Fri'): 7,

    ('Ai', 'Mon'): 28,
    ('Ai', 'Tue'): 24,
    ('Ai', 'Wed'): 22,
    ('Ai', 'Thu'): 26,
    ('Ai', 'Fri'): 25,

    ('An', 'Mon'): 10,
    ('An', 'Tue'): 6,
    ('An', 'Wed'): 7,
    ('An', 'Thu'): 8,
    ('An', 'Fri'): 5,

    ('Ni', 'Mon'): 28,
    ('Ni', 'Tue'): 24,
    ('Ni', 'Wed'): 22,
    ('Ni', 'Thu'): 26,
    ('Ni', 'Fri'): 25,

    ('Nn', 'Mon'): 12,
    ('Nn', 'Tue'): 5,
    ('Nn', 'Wed'): 8,
    ('Nn', 'Thu'): 6,
    ('Nn', 'Fri'): 7
}

# We check the length of matrix
# to be equal to 15 (3 x 5)
assert len(patient_forecast.items()) == len(WEEK)*len(SHIFTS)

# We define the forescated number of patients per scenario and
# days of the week into the model. This information is
# crucial in the solution to the problem
nonlinear_model.patients = pyo.Param(nonlinear_model.i, nonlinear_model.j,
    initialize = patient_forecast, doc = "Forescasted patients per scenario and day of the week")

# Print the basic parts of the continuous model
nonlinear_model.pprint()

3 Set Declarations
    i : scenarios
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'Mi', 'Mn', 'Ai', 'An', 'Ni', 'Nn'}
    j : days of the week
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'Mon', 'Tue', 'Wed', 'Thu', 'Fri'}
    patients_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   30 : {('Mi', 'Mon'), ('Mi', 'Tue'), ('Mi', 'Wed'), ('Mi', 'Thu'), ('Mi', 'Fri'), ('Mn', 'Mon'), ('Mn', 'Tue'), ('Mn', 'Wed'), ('Mn', 'Thu'), ('Mn', 'Fri'), ('Ai', 'Mon'), ('Ai', 'Tue'), ('Ai', 'Wed'), ('Ai', 'Thu'), ('Ai', 'Fri'), ('An', 'Mon'), ('An', 'Tue'), ('An', 'Wed'), ('An', 'Thu'), ('An', 'Fri'), ('Ni', 'Mon'), ('Ni', 'Tue'), ('Ni', 'Wed'), ('Ni', 'Thu'), ('Ni', 'Fri'), ('Nn', 'Mon'), ('Nn', 'Tue'), ('Nn', 'Wed'), ('Nn', 'Thu'), ('Nn', 'Fri')}

1 Param D

In [62]:
# We define variables x(i,j)
# that represent the number of intensivists
# per scenario, per day of the week
nonlinear_model.x = pyo.Var(nonlinear_model.i, nonlinear_model.j,
    doc = "Number of intensivists",
    within = pyo.NonNegativeReals)

print(nonlinear_model.x.display())

x : Number of intensivists
    Size=30, Index=x_index
    Key           : Lower : Value : Upper : Fixed : Stale : Domain
    ('Ai', 'Fri') :     0 :  None :  None : False :  True : NonNegativeReals
    ('Ai', 'Mon') :     0 :  None :  None : False :  True : NonNegativeReals
    ('Ai', 'Thu') :     0 :  None :  None : False :  True : NonNegativeReals
    ('Ai', 'Tue') :     0 :  None :  None : False :  True : NonNegativeReals
    ('Ai', 'Wed') :     0 :  None :  None : False :  True : NonNegativeReals
    ('An', 'Fri') :     0 :  None :  None : False :  True : NonNegativeReals
    ('An', 'Mon') :     0 :  None :  None : False :  True : NonNegativeReals
    ('An', 'Thu') :     0 :  None :  None : False :  True : NonNegativeReals
    ('An', 'Tue') :     0 :  None :  None : False :  True : NonNegativeReals
    ('An', 'Wed') :     0 :  None :  None : False :  True : NonNegativeReals
    ('Mi', 'Fri') :     0 :  None :  None : False :  True : NonNegativeReals
    ('Mi', 'Mon') :     0 :  Non

In [63]:
# We define the objective function to be used
def nl_objective_rule(nl_model: pyo.ConcreteModel) -> float:
    return sum([((nl_model.x[i,j]/nl_model.patients[i,j])-(2/15))**2 for count, i in enumerate(nl_model.i) for j in nl_model.j])

In [64]:
# We instantiate the objective function for the model which is passed as a rule
# although Pyomo minimizes by default, we still explicitly specify this
nonlinear_model.objective = pyo.Objective(
                                            rule = nl_objective_rule,
                                            sense = pyo.minimize,
                                             doc = "Real-to-Optimal Intensivist-to-Patient ratio"
                                             )

# Visualize the objective function
nonlinear_model.objective.pprint()

objective : Real-to-Optimal Intensivist-to-Patient ratio
    Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : (0.04*x[Mi,Mon] - 0.13333333333333333)**2 + (0.045454545454545456*x[Mi,Tue] - 0.13333333333333333)**2 + (0.05555555555555555*x[Mi,Wed] - 0.13333333333333333)**2 + (0.043478260869565216*x[Mi,Thu] - 0.13333333333333333)**2 + (0.03333333333333333*x[Mi,Fri] - 0.13333333333333333)**2 + (0.06666666666666667*x[Mn,Mon] - 0.13333333333333333)**2 + (0.125*x[Mn,Tue] - 0.13333333333333333)**2 + (0.1*x[Mn,Wed] - 0.13333333333333333)**2 + (0.125*x[Mn,Thu] - 0.13333333333333333)**2 + (0.14285714285714285*x[Mn,Fri] - 0.13333333333333333)**2 + (0.03571428571428571*x[Ai,Mon] - 0.13333333333333333)**2 + (0.041666666666666664*x[Ai,Tue] - 0.13333333333333333)**2 + (0.045454545454545456*x[Ai,Wed] - 0.13333333333333333)**2 + (0.038461538461538464*x[Ai,Thu] - 0.13333333333333333)**2 + (0.04*x[Ai,Fri] - 0.13333333333333333)**2 + (0.1*x[An,Mon] - 0

In [65]:
# Function that establishes the constraint by which the solution to the number of doctors
# cannot be larger than the number of available medical staff
def nl_MaxStaffPerDay(nl_model: pyo.ConcreteModel, j: str) -> bool:
    return sum(nl_model.x[i,j] for i in nl_model.i) <= NUMBER_OF_INTENSIVISTS

In [66]:
# This constraint is now added to the model
nonlinear_model.maxStaffPerDay = pyo.Constraint(
    nonlinear_model.j,
    rule = nl_MaxStaffPerDay,
    doc = "Maximum number of doctors per day")

# Visualize the constraint maxStaffPerDay
nonlinear_model.maxStaffPerDay.pprint()

maxStaffPerDay : Maximum number of doctors per day
    Size=5, Index=j, Active=True
    Key : Lower : Body                                                                  : Upper : Active
    Fri :  -Inf : x[Mi,Fri] + x[Mn,Fri] + x[Ai,Fri] + x[An,Fri] + x[Ni,Fri] + x[Nn,Fri] :  24.0 :   True
    Mon :  -Inf : x[Mi,Mon] + x[Mn,Mon] + x[Ai,Mon] + x[An,Mon] + x[Ni,Mon] + x[Nn,Mon] :  24.0 :   True
    Thu :  -Inf : x[Mi,Thu] + x[Mn,Thu] + x[Ai,Thu] + x[An,Thu] + x[Ni,Thu] + x[Nn,Thu] :  24.0 :   True
    Tue :  -Inf : x[Mi,Tue] + x[Mn,Tue] + x[Ai,Tue] + x[An,Tue] + x[Ni,Tue] + x[Nn,Tue] :  24.0 :   True
    Wed :  -Inf : x[Mi,Wed] + x[Mn,Wed] + x[Ai,Wed] + x[An,Wed] + x[Ni,Wed] + x[Nn,Wed] :  24.0 :   True


In [67]:
# Function that implements the minimal Intensivist-to-Patients ratio that
# is minimally admissible taking into account healthcare standards
# for Level I (which only requires physicians during weekdays)
def nl_MinProportion(nl_model: pyo.ConcreteModel, constraintList: pyo.ConstraintList) -> None:
    for i in SHIFTS:
        for j in WEEK:
            constraintList.add(expr = nl_model.x[i,j]/nl_model.patients[i,j] >= 1/14)

    assert len(constraintList) == len(SHIFTS)*len(WEEK)
    constraintList.pprint()
    
    return

In [68]:
# Function that implements the minimal Intensivist-to-Patients ratio that
# is minimally admissible taking into account healthcare standards
# for Level I (which only requires physicians during weekdays)
def nl_MaxProportion(nl_model: pyo.ConcreteModel, constraintList: pyo.ConstraintList) -> None:
    for i in SHIFTS:
        for j in WEEK:
            constraintList.add(expr = nl_model.x[i,j]/nl_model.patients[i,j] <= 1/3)

    assert len(constraintList) == len(SHIFTS)*len(WEEK)
    constraintList.pprint()
    
    return

In [69]:
nonlinear_model.minproportions = pyo.ConstraintList()
nl_MinProportion(nonlinear_model, nonlinear_model.minproportions)

minproportions : Size=30, Index=minproportions_index, Active=True
    Key : Lower               : Body                           : Upper : Active
      1 : 0.07142857142857142 :                 0.04*x[Mi,Mon] :  +Inf :   True
      2 : 0.07142857142857142 : 0.045454545454545456*x[Mi,Tue] :  +Inf :   True
      3 : 0.07142857142857142 :  0.05555555555555555*x[Mi,Wed] :  +Inf :   True
      4 : 0.07142857142857142 : 0.043478260869565216*x[Mi,Thu] :  +Inf :   True
      5 : 0.07142857142857142 :  0.03333333333333333*x[Mi,Fri] :  +Inf :   True
      6 : 0.07142857142857142 :  0.06666666666666667*x[Mn,Mon] :  +Inf :   True
      7 : 0.07142857142857142 :                0.125*x[Mn,Tue] :  +Inf :   True
      8 : 0.07142857142857142 :                  0.1*x[Mn,Wed] :  +Inf :   True
      9 : 0.07142857142857142 :                0.125*x[Mn,Thu] :  +Inf :   True
     10 : 0.07142857142857142 :  0.14285714285714285*x[Mn,Fri] :  +Inf :   True
     11 : 0.07142857142857142 :  0.03571428571428571*x

In [70]:
nonlinear_model.maxproportions = pyo.ConstraintList()
nl_MaxProportion(nonlinear_model, nonlinear_model.maxproportions)

maxproportions : Size=30, Index=maxproportions_index, Active=True
    Key : Lower : Body                           : Upper              : Active
      1 :  -Inf :                 0.04*x[Mi,Mon] : 0.3333333333333333 :   True
      2 :  -Inf : 0.045454545454545456*x[Mi,Tue] : 0.3333333333333333 :   True
      3 :  -Inf :  0.05555555555555555*x[Mi,Wed] : 0.3333333333333333 :   True
      4 :  -Inf : 0.043478260869565216*x[Mi,Thu] : 0.3333333333333333 :   True
      5 :  -Inf :  0.03333333333333333*x[Mi,Fri] : 0.3333333333333333 :   True
      6 :  -Inf :  0.06666666666666667*x[Mn,Mon] : 0.3333333333333333 :   True
      7 :  -Inf :                0.125*x[Mn,Tue] : 0.3333333333333333 :   True
      8 :  -Inf :                  0.1*x[Mn,Wed] : 0.3333333333333333 :   True
      9 :  -Inf :                0.125*x[Mn,Thu] : 0.3333333333333333 :   True
     10 :  -Inf :  0.14285714285714285*x[Mn,Fri] : 0.3333333333333333 :   True
     11 :  -Inf :  0.03571428571428571*x[Ai,Mon] : 0.333333333333

In [71]:
Solver = SolverFactory('ipopt')
nonlinear_model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
Results = Solver.solve(nonlinear_model, tee=True)
print(Results)

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       90
Number of nonzeros in Lagrangian Hessian.............:       30

Total number of variables............................:       30
                     variables with only lower bounds:       30
                variables with lower and upper bounds:        0


In [72]:
for i in SHIFTS:
    for j in WEEK:
        print(f"x[{i},{j}] = {pyo.value(nonlinear_model.x[i,j])}")

x[Mi,Mon] = 3.3333335109439846
x[Mi,Tue] = 2.933333501741132
x[Mi,Wed] = 2.4000001407467813
x[Mi,Thu] = 3.0666668399574353
x[Mi,Fri] = 4.00000021649298
x[Mn,Mon] = 2.000000114814607
x[Mn,Tue] = 1.0666667321026921
x[Mn,Wed] = 1.33333341445925
x[Mn,Thu] = 1.06666673187153
x[Mn,Fri] = 0.9333333906256159
x[Ai,Mon] = 3.733333527638452
x[Ai,Tue] = 3.2000001819192674
x[Ai,Wed] = 2.93333350213103
x[Ai,Thu] = 3.4666668593567054
x[Ai,Fri] = 3.3333335190052495
x[An,Mon] = 1.3333334126261314
x[An,Tue] = 0.800000049526948
x[An,Wed] = 0.9333333908917228
x[An,Thu] = 1.06666673187153
x[An,Fri] = 0.6666667080109975
x[Ni,Mon] = 3.733333527638452
x[Ni,Tue] = 3.2000001819192674
x[Ni,Wed] = 2.93333350213103
x[Ni,Thu] = 3.4666668593567054
x[Ni,Fri] = 3.3333335190052495
x[Nn,Mon] = 1.6000000938314338
x[Nn,Tue] = 0.666666708126657
x[Nn,Wed] = 1.0666667321542054
x[Nn,Thu] = 0.8000000493969075
x[Nn,Fri] = 0.9333333906256159


In [73]:
print(pyo.value(nonlinear_model.objective))

1.818015766190091e-15


In [79]:
nonlinear_model.dual.display()

dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                 : Value
    maxStaffPerDay[Fri] :  -8.416592970777641e-11
    maxStaffPerDay[Mon] : -1.0996143679087478e-10
    maxStaffPerDay[Thu] :  -8.213761749529855e-11
    maxStaffPerDay[Tue] :   -7.49159362248133e-11
    maxStaffPerDay[Wed] :  -7.330464581746253e-11
     maxproportions[10] :   -4.54544216149583e-09
     maxproportions[11] :  -4.545443968452732e-09
     maxproportions[12] :   -4.54544371326544e-09
     maxproportions[13] :  -4.545443655199715e-09
     maxproportions[14] :  -4.545443830241727e-09
     maxproportions[15] : -4.5454438006896796e-09
     maxproportions[16] :  -4.545442971391676e-09
     maxproportions[17] :  -4.545441851457744e-09
     maxproportions[18] :  -4.545442151468265e-09
     maxproportions[19] :  -4.545442395086244e-09
      maxproportions[1] :  -4.545443933639608e-09
     maxproportions[20] :  -4.545441601748174e-09
     maxproportions[21] :  -4.545443968452732e-09
     maxpropor